In [1]:
import pandas as pd
import numpy as np
import random
import sys
sys.path.append('../..')
from modules.many_features import utils, constants
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

C:\Users\User\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning:

`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:167: DeprecationWarning:

`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `

In [2]:
all_df= pd.read_csv('../../data/more_features/more_feats_correlated_0.1.csv')
#df = df.fillna(-1)
all_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,11.183192,187.573466,4.951674,1.661383,316.593436,95.006424,181.242992,3.531295,1,NaN,28.040619,96.721542,49.530542,23.186628,NaN,33.549575,57.247868,Hemolytic anemia
1,14.387445,NaN,NaN,NaN,NaN,NaN,125.249617,NaN,1,NaN,98.357508,112.758764,62.464566,7.168920,NaN,43.162335,NaN,No anemia
2,12.749357,5.012158,3.502800,6.179371,498.418768,76.759285,159.834784,4.982859,1,1.022939,56.850479,75.739552,72.072041,20.600875,44.872138,38.248071,32.068372,Iron deficiency anemia
3,11.508870,197.180945,1.200125,0.000000,457.033309,102.900301,131.177927,3.355346,1,NaN,111.220307,66.999185,18.353272,14.132423,NaN,34.526609,28.702050,Unspecified anemia
4,9.456656,427.952052,NaN,0.660252,NaN,104.543774,NaN,2.713693,0,NaN,NaN,NaN,NaN,NaN,NaN,28.369968,NaN,Vitamin B12/Folate deficiency anemia


In [3]:
len(all_df)

70000

In [4]:
class_dict = constants.CLASS_DICT
all_df['label'] = all_df['label'].replace(class_dict)
X = all_df.iloc[:, 0:-1]
y = all_df.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
X_train_df = pd.concat([X_train, y_train], axis=1)
X_test_df = pd.concat([X_test, y_test], axis=1)
X_test_df = X_test_df.reset_index(drop=True)
X_test_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,11.391136,NaN,1.504298,5.058881,467.197112,96.252926,217.860499,3.550376,1,NaN,36.703810,122.310168,49.897027,20.353251,NaN,34.173407,46.631388,6
1,10.355048,272.506966,NaN,0.000000,128.706486,101.912313,NaN,3.048223,1,NaN,NaN,NaN,29.327349,NaN,NaN,31.065144,NaN,2
2,11.159385,NaN,NaN,0.000000,NaN,103.395616,77.212369,3.237870,0,NaN,8.690122,65.554731,19.167967,7.239049,NaN,33.478155,NaN,2
3,8.179735,140.876632,5.484515,NaN,303.740826,99.877458,155.573175,2.456931,0,NaN,41.123526,119.433840,39.384848,6.394235,NaN,24.539204,51.219053,5
4,9.916825,NaN,3.909810,NaN,NaN,90.543986,189.347916,3.285748,0,1.501433,34.905667,59.177001,58.538524,22.843594,139.245204,29.750475,NaN,5


In [5]:
def insert_nans(df, col, frac):
    non_null_indices = df[df[col].notna()].index.tolist() #getting not null indices
    to_change_indices = random.sample(non_null_indices, int(frac*len(non_null_indices)))
    df.loc[to_change_indices, col] = np.nan
    return df

In [6]:
frac = 0.3

In [7]:
for column in X_train_df.columns:
    if column not in ['hemoglobin', 'gender', 'label']:
        X_train_df = insert_nans(X_train_df, column, frac)
X_train_df.isna().sum()

hemoglobin                   0
ferritin                 30832
ret_count                30779
segmented_neutrophils    29280
tibc                     31288
mcv                      19590
serum_iron               31269
rbc                      19590
gender                       0
creatinine               38334
cholestrol               31269
copper                   31269
ethanol                  27755
folate                   31269
glucose                  45404
hematocrit               16800
tsat                     40394
label                        0
dtype: int64

In [8]:
X_train_df.shape

(56000, 18)

In [9]:
utils.get_dt_performance(X_train_df, 'numeric')

(0.7729464285714286,
 0.7402227633753703,
 0.8538540681726781,
 datetime.timedelta(microseconds=4270))

In [10]:
X_train_df.to_csv('../../data/more_features/train_sets/train_set_missing_3.csv', index=False)